In [4]:
# Unimi 2023/2024
# Progetto di Visualizzazione Scientifica
# 984605 Pomayay Gabonal Angello Fernando 
# 01553A Mbaye Serigne Darou

import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output

# Selezione range anni
years = range(1977, 2025)

# Creazione dell'app dash
app = dash.Dash(__name__)

# Definizione del layout
app.layout = html.Div([
    html.H1("Top 10 Film per anno"),
    html.Label("Seleziona l'anno:"),
    dcc.Slider(
        id='year-slider',
        min=min(years),
        max=max(years),
        value=max(years),
        marks={year: str(year) for year in years},
        step=None
    ),
    dcc.Graph(id='top-movies-graph')
])


@app.callback(
    Output('top-movies-graph', 'figure'),
    [Input('year-slider', 'value')]
)
def update_graph(selected_year):
    # Acquisizione del file CSV in base all'anno
    file_path = f'../csv_guadagni/ranking_summary_{selected_year}.csv'
    df_data = pd.read_csv(file_path)

    # Conversione dei valori della colonna worldwide in valori numerici
    df_data['worldwide'] = df_data['worldwide'].replace('[\$,]', '', regex=True).astype(float)

    # Ordinamento crescente dei guadagni mondiali nel DataFrame
    df_data = df_data.sort_values(by='worldwide', ascending=True)

    # Selezione dei top 10 film
    top_10_movies = df_data.tail(10)

    # Creazione del grafico
    fig = px.bar(
        top_10_movies,
        x='worldwide',
        y='title',
        color='rank',
        labels={'worldwide': 'Guadagno al botteghino (Milioni)', 'title': 'Titolo Film'},
        title=f'Top 10 Film del {selected_year} - Guadagno al botteghino mondiale',
        color_discrete_sequence=px.colors.sequential.Viridis,
        height=800,
        width=1800
    )

    fig.update_traces(marker=dict(line=dict(width=2)))

    # Aggiunta hover
    fig.update_traces(
        hovertemplate='Earnings: %{x:.2f}B' if top_10_movies['worldwide'].max() >= 1e9 else 'Earnings: %{x:.2f}M',
        selector=dict(type='bar')
    )

    
    fig.update_layout(
        legend=dict(orientation="v", yanchor="top", y=1.02, xanchor="right", x=1.1),
        legend_traceorder="reversed",
        plot_bgcolor='rgba(255, 255, 255, 0)',
        paper_bgcolor='rgba(255, 255, 255, 0)',
        font_color='black'
    )

    return fig


if __name__ == '__main__':
    port = 8050  
    app.run_server(debug=True, port=port)
    
    print(f"Il server è attivo all'indirizzo: http://127.0.0.1:{port}/")


Il server è attivo all'indirizzo: http://127.0.0.1:8050/
